#### TESTING GROQ With LLAMA INDEX

In [16]:
import chromadb
from llama_index.llms.groq import Groq
import chromadb
import os

In [17]:
GROK_API_KEY = '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'
LLAMA_HUB_API_KEY = 'llx-VbW7L7fNit8D5IdAKWGhDg1fzzc1SJUPiHQzqLRysQxRx5tQ'

In [18]:
llm = Groq(model="llama-3.2-3b-preview", api_key=GROK_API_KEY)

In [5]:
response = llm.complete("Explain the importance of low latency LLMs")

In [6]:
print(response)

Low latency Large Language Models (LLMs) are crucial in various applications where real-time processing and response are essential. Here are some reasons why low latency LLMs are important:

1. **Real-time Conversations**: In applications like chatbots, virtual assistants, and customer service platforms, low latency LLMs enable seamless and responsive conversations. Users expect immediate responses, and low latency ensures that the model can provide accurate and relevant information in a timely manner.
2. **Gaming and Interactive Applications**: In games, interactive simulations, and other immersive experiences, low latency LLMs are vital for a smooth and engaging user experience. Fast response times allow for real-time decision-making, reducing lag and improving overall player satisfaction.
3. **Financial Applications**: In high-stakes financial applications like trading platforms, stock analysis, and risk management, low latency LLMs are critical for making quick and informed decisio

In [19]:
from llama_index.readers.smart_pdf_loader import SmartPDFLoader
import nest_asyncio

nest_asyncio.apply()

from llama_parse import LlamaParse

parser = LlamaParse(
    api_key=LLAMA_HUB_API_KEY,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    # result_type="markdown",  # "markdown" and "text" are available
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
    language="en",  # Optionally you can define a language, default=en
)



In [ ]:

# sync
documents = parser.load_data("/Users/emmanuel/hackathon/docs/o-and-m-best-practice-guidelines-v-5-0-a07c44238b.pdf")
# documents = pdf_loader.load_data("/Users/emmanuel/hackathon/docs/o-and-m-best-practice-guidelines-v-5-0-a07c44238b.pdf")

Started parsing the file under job_id cd3839c7-652b-401f-bf06-857eac152d23
......................

In [20]:
second_document = parser.load_data("/Users/emmanuel/hackathon/docs/Guide_to_Solar_Optimization-Ark_Resources.pdf")
third_document = parser.load_data("/Users/emmanuel/hackathon/docs/Plant-documentation_ Information-advice-Menger.pdf")

INFO:httpx:HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"
Started parsing the file under job_id 49a760b1-6e35-4f00-8195-ef9d01eadf73
INFO:httpx:HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/49a760b1-6e35-4f00-8195-ef9d01eadf73 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/49a760b1-6e35-4f00-8195-ef9d01eadf73 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/49a760b1-6e35-4f00-8195-ef9d01eadf73 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/49a760b1-6e35-4f00-8195-ef9d01eadf73 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/49a760b1-6e35-4f00-8195-ef9d01eadf73 "HTTP/1.1 200 OK"
HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/49a760b1-6e35-4f00-8195-ef9d01eadf

In [11]:
import chromadb
from chromadb.config import Settings
client_ = chromadb.PersistentClient(path="./embeds")

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [ ]:
from typing import List,Any, Tuple
from sentence_transformers import SentenceTransformer


def parse_document(file_path, parser):
    """Accepts both a file path and a list of file paths"""
    documents = parser.load_data(file_path)
    return documents

def retrieve_similar(query:str, client: chromadb.Client, collection_name:str="constructionom",  n_results=2): #Maybe add type hints later
    """Retrieves similar documents from a collection"""
    collection = client.get_or_create_collection(name=collection_name)
    results = collection.query( 
        query_texts=[query], # Chroma will embed this for you #can query embedding too
        n_results=n_results # how many results to return
    )
    return results['documents']

def prep_doc_for_upsert_document(documents:List[Any], embedding_model = SentenceTransformer()):
    """Upserts a document to the embedding store"""

    embeddings = []
    ids = []
    text = []

    for doc in documents:
        embeddings.append(embedding_model.encode(doc.text))
        ids.append(doc.doc_id)
        text.append(doc.text)
    return (ids,text)

def upsert_embeddings(client:chromadb.Client, document:Tuple[List], embeddings, collection_name:str="constructionom"):
    collection = client.get_or_create_collection(name=collection_name)
    collection.upsert(embeddings=document[2], documents=document[1], ids = document[0])
    print("upsert done")


#### Agentic Workflow with LLAMA

### BASE EMBEDDINGS WITH SentenceTransformers , all-MiniLM-L6-v2'


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [36]:
for doc in documents:
    doc.embedding = model.encode(doc.text)

### FINETUNE EMBEDDING MODELS

In [1]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from llama_index.core.node_parser import SentenceSplitter

In [45]:
from llama_index.core import SimpleDirectoryReader
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SentenceSplitter()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes


In [48]:
os.listdir('docs')

['Guide_to_Solar_Optimization-Ark_Resources.pdf',
 'Plant-documentation_ Information-advice-Menger.pdf',
 'o-and-m-best-practice-guidelines-v-5-0-a07c44238b.pdf']

In [57]:
train_nodes = load_corpus(["/Users/emmanuel/hackathon/docs/" + i for i in  os.listdir('docs')])
val_nodes = load_corpus(["/Users/emmanuel/hackathon/test_docs/" + i  for i in  os.listdir('test_docs')])

In [ ]:

from llama_index.llms.openai import OpenAI

train_dataset = generate_qa_embedding_pairs(
    llm=llm,
    nodes=train_nodes,
    output_path="train_dataset.json",
)
val_dataset = generate_qa_embedding_pairs(
    llm=llm,
    nodes=val_nodes,
    output_path="val_dataset.json",
)

In [2]:
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
val_dataset = EmbeddingQAFinetuneDataset.from_json("val_dataset.json")

In [3]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine
finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    model_id="BAAI/bge-small-en",
    model_output_path="test_model",
    val_dataset=val_dataset,
)

/Users/emmanuel/.virtualenvs/hackathon-cmtz/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:datasets:PyTorch version 2.5.1 available.
PyTorch version 2.5.1 available.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en


In [5]:
finetune_engine.finetune()
embed_model = finetune_engine.get_finetuned_model()

 50%|█████     | 29/58 [00:37<00:35,  1.23s/it]

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Information Retrieval Evaluation of the model on the  dataset after epoch 1.0:
Information Retrieval Evaluation of the model on the  dataset after epoch 1.0:
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Queries: 32
Queries: 32
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Corpus: 16

Corpus: 16

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Score-Function: cosine
Score-Function: cosine
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@1: 43.75%
Accuracy@1: 43.75%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@3: 56.25%
Accuracy@3: 56.25%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@5: 65.62%
Accuracy@5: 65.62%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@10: 78.12%
Accuracy@10: 78.12%
INFO:sentence_transformers.evaluation.InformationR

 86%|████████▌ | 50/58 [01:04<00:10,  1.26s/it]

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Information Retrieval Evaluation of the model on the  dataset:
Information Retrieval Evaluation of the model on the  dataset:
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Queries: 32
Queries: 32
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Corpus: 16

Corpus: 16

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Score-Function: cosine
Score-Function: cosine
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@1: 43.75%
Accuracy@1: 43.75%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@3: 56.25%
Accuracy@3: 56.25%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@5: 65.62%
Accuracy@5: 65.62%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@10: 81.25%
Accuracy@10: 81.25%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Precision@1: 4

 86%|████████▌ | 50/58 [01:05<00:10,  1.26s/it]

...

100%|██████████| 58/58 [01:15<00:00,  1.21s/it]

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Information Retrieval Evaluation of the model on the  dataset after epoch 2.0:
Information Retrieval Evaluation of the model on the  dataset after epoch 2.0:
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Queries: 32
Queries: 32
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Corpus: 16

Corpus: 16

INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Score-Function: cosine
Score-Function: cosine
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@1: 43.75%
Accuracy@1: 43.75%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@3: 56.25%
Accuracy@3: 56.25%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@5: 65.62%
Accuracy@5: 65.62%
INFO:sentence_transformers.evaluation.InformationRetrievalEvaluator:Accuracy@10: 81.25%
Accuracy@10: 81.25%
INFO:sentence_transformers.evaluation.InformationR

100%|██████████| 58/58 [01:15<00:00,  1.31s/it]

{'train_runtime': 75.8673, 'train_samples_per_second': 7.592, 'train_steps_per_second': 0.764, 'train_loss': 1.3479036791571255, 'epoch': 2.0}
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: test_model
Load pretrained SentenceTransformer: test_model


INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [56]:
temp_emb = []
for doc in third_document:
    xy = await embed_model.aget_text_embedding(doc.text[0])
    temp_emb.append(xy)

Batches: 100%|██████████| 1/1 [00:00<00:00, 54.57it/s]


### SAVE EMBEDDINGS WITH CHROMA

In [67]:
upsert_embeddings(client=client_, embeddings=temp_emb, document=prep_doc_for_upsert_document(third_document))

upsert done


In [ ]:
# obj = await embed_model.aget_text_embedding("Tell me about solar panels installation in europe")

In [ ]:
# def PersistentClient(path: str = "./chroma",
#                      settings: Optional[Settings] = None,
#                      tenant: str = DEFAULT_TENANT,
#                      database: str = DEFAULT_DATABASE) -> ClientAPI

#prod

In [69]:
question= "Tell me about solar panels installation in europe"
context = retrieve_similar(client=client_, query=question)

context_answer = llm.complete(f"""Using only {context} as context, 
                              can you please provide detailed instruction this question? {question}. 
                              If the context {context}  does not correspond with the question {question}, please say you do not know""")
print(f"Context: {context}")
print(context_answer.text)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Context: [['SolarPower Europe would like to thank the members of its Lifecycle\nQuality Workstream that contributed to this report including:\n                                                                  WIND\n\n\n\n\n\n\n                                                                   ENPHASE\n      everoze                                                       Fraunhofer\n                                                                               CSP\n                                            3  Solar\n                                                                  Qe n e r G Y\n                             Sonnedix\n                                                  ENERGY\n                                                        WISE\n     Sponsor members of SolarPower Europe:\n             Ba

- Reflection Agent which asseses the prompt in relation to the context

In [42]:
non_context_answer = llm.complete(f"""Using your knowledge base. 
                              can you please provide detailed instruction this question? {question}. 
                              If it isn't present in your knowledge base, please say you do not know""")
print(non_context_answer.text)

I can provide you with detailed instructions on solar panel installation in Europe. Here's a comprehensive guide:

**Introduction**

Solar panel installation is a popular renewable energy source in Europe, with many countries setting ambitious targets to reduce greenhouse gas emissions and increase the use of renewable energy. In this guide, we'll cover the key steps involved in installing solar panels in Europe.

**Pre-Installation Requirements**

Before installing solar panels, you'll need to:

1. **Check local regulations**: Familiarize yourself with local building codes, zoning laws, and regulations regarding solar panel installation. Some countries have specific requirements for solar panel installations, such as minimum roof size or orientation.
2. **Assess your energy needs**: Determine your energy consumption and assess whether solar panels can meet your energy needs. Consider factors like your energy usage patterns, roof size, and shading.
3. **Choose a suitable location**: Se

In [ ]:
#collection. peek